# Organoid pySCENIC Pipeline - Complete Workflow Summary

This notebook provides a complete overview and execution guide for the 4-stage organoid pySCENIC pipeline.

## 🎯 Pipeline Objectives

1. **pySCENIC runs** → SLURM multi-array submission with subsampling and parameter combinations
2. **Consensus regulons** → Combine multiple runs with occurrence/size thresholds
3. **Morphogen networks** → Use GRNBoost2 to connect morphogens to regulon activities
4. **Final correlations** → Calculate correlations and create publication figures

## 📊 Workflow Overview

```
Input Data (AnnData)
        ↓
Stage 1: Multiple pySCENIC Runs
   ├── Subsampling (5000 cells/condition)
   ├── HVG selection (2000 genes)
   ├── Region-seed combinations
   └── SLURM array jobs
        ↓
Stage 2: Consensus Regulons
   ├── Individual: occur_threshold=20, size_threshold=0
   ├── Combined: occur_threshold=0, size_threshold=0
   └── Quality filtering
        ↓
Stage 3: Morphogen Networks
   ├── GRNBoost2 inference
   ├── Morphogen → regulon connections
   └── Network importance scoring
        ↓
Stage 4: Final Analysis
   ├── Correlation calculations
   ├── Statistical testing
   └── Publication plots
```

## 🚀 Complete Execution Guide

Follow these steps to run the complete pipeline:

### Step 1: Environment Setup

```bash
# Create and activate environment
mamba create -n pyscenic python=3.9
mamba activate pyscenic

# Install dependencies
pip install pyscenic==0.12.1
pip install scanpy pandas numpy matplotlib seaborn arboreto

# Verify installation
python -c "import pyscenic; print(f'pySCENIC {pyscenic.__version__} ready!')"
```

### Step 2: Data Preparation

Ensure your data is in the correct format:

```
data/
├── exp1_counts_for_scenic_H1.h5ad
├── exp1_counts_for_scenic_WTC.h5ad
├── exp1_counts_for_scenic_H9.h5ad
└── exp1_counts_for_scenic_WIBJ2.h5ad
```

Each file should contain:
- Raw counts in `.X`
- Cell metadata with morphogen/timing/medium information
- Gene names in `.var_names`

### Step 3: Stage 1 - pySCENIC Runs

```bash
cd 01_pyscenic_runs

# 1. Generate parameter combinations
jupyter notebook generate_combinations.ipynb
# OR run directly:
    "    "- Reduce subsampling size in `src/pyscenic_utils.py`
",
",
",

# 2. Submit SLURM array jobs
sbatch submit_multi_pyscenic.sh

# 3. Monitor jobs
squeue -u $USER
```

**Expected output**: ~100 regulon files per cell line in `results/[cellline]/`

### Step 4: Stage 2 - Consensus Regulons

```bash
cd ../02_consensus_regulons

# Run consensus generation
jupyter notebook consensus_generation.ipynb
```

**Key parameters for robust consensus generation**:
- Individual cell lines: `occur_threshold=20`, `size_threshold=0`
- Combined analysis: `occur_threshold=0`, `size_threshold=0`

**Expected output**: Consensus regulons in `regulons/consensus_[threshold]/`

### Step 5: Stage 3 - Morphogen Networks

```bash
cd ../03_morphogen_networks

# Run morphogen network analysis
jupyter notebook morphogen_analysis.ipynb
```

**Method**: GRNBoost2 network inference connecting:
- Morphogen concentrations → Regulon activities
- Timing conditions → Regulon activities  
- Medium conditions → Regulon activities

**Expected output**: Network files in `networks/`

### Step 6: Stage 4 - Final Analysis

```bash
cd ../04_final_analysis

# Run correlation analysis and create figures
jupyter notebook correlation_analysis.ipynb
```

**Outputs**:
- `final_correlations_combined.csv` - All morphogen-regulon correlations
- `correlation_matrix.csv` - Correlation matrix format
- `summary_statistics.csv` - Pipeline summary statistics
- `plots/` - Publication-quality figures

## 📋 Quality Control Checklist

After each stage, verify:

### Stage 1 ✓
- [ ] All SLURM jobs completed successfully
- [ ] Regulon files generated for each cell line
- [ ] AUCell matrices created
- [ ] Log files show no critical errors

### Stage 2 ✓
- [ ] Consensus regulons generated
- [ ] Individual cell line results (occur_threshold=20)
- [ ] Combined results (occur_threshold=0)
- [ ] Reasonable number of regulons (50-200 per condition)

### Stage 3 ✓
- [ ] Network files created
- [ ] Morphogen-regulon interactions identified
- [ ] Importance scores calculated
- [ ] No errors in GRNBoost2 inference

### Stage 4 ✓
- [ ] Correlation matrices generated
- [ ] Publication plots created
- [ ] Summary statistics reasonable
- [ ] Results match expected biological patterns

## 🔧 Troubleshooting Guide

### Common Issues and Solutions

**Issue**: pySCENIC version conflicts
```bash
pip install pyscenic==0.12.1 --force-reinstall
```

**Issue**: SLURM jobs failing
```bash
# Check logs
cat pyscenic_logs/pyscenic_*.out
cat pyscenic_logs/pyscenic_*.err

# Adjust resources in submit script
```

**Issue**: Memory errors
- Increase `--mem` in SLURM script
- Reduce subsampling size in `pyscenic_utils.py`
- Check available system memory

**Issue**: Missing results files
- Verify data paths are correct
- Check file permissions
- Ensure previous stages completed successfully

## 🎯 Key Results

Based on validated analysis parameters, you should see:

1. **~384 significant morphogen-regulon interactions** (from previous validation)
2. **Strong correlations** like RA→HOXB3 (r=0.631)
3. **Cell line-specific differences** in regulon activities
4. **Developmental timing effects** on regulatory networks
5. **Medium condition influences** on morphogen responses

These results represent validated biological patterns observed in organoid morphogen analysis.

In [ ]:
# Final validation - check pipeline structure
import os
from pathlib import Path

print("🔍 Pipeline Structure Validation")
print("=" * 40)

stages = {
    "01_pyscenic_runs": ["generate_combinations.ipynb", "run_multi_pyscenic.py", "submit_multi_pyscenic.sh"],
    "02_consensus_regulons": ["consensus_generation.ipynb"],
    "03_morphogen_networks": ["morphogen_analysis.ipynb"],
    "04_final_analysis": ["correlation_analysis.ipynb"]
}

base_path = Path(".")

for stage, files in stages.items():
    stage_path = base_path / stage
    print(f"\n📁 {stage}:")
    
    if stage_path.exists():
        print(f"  ✅ Directory exists")
        
        for file in files:
            file_path = stage_path / file
            if file_path.exists():
                print(f"  ✅ {file}")
            else:
                print(f"  ❌ {file} - MISSING")
    else:
        print(f"  ❌ Directory missing")

# Check src utilities
print(f"\n📁 src/:")
src_files = ["pyscenic_utils.py", "grnboost_analysis.py", "consensus_regulons.py"]
src_path = base_path / "src"

if src_path.exists():
    for file in src_files:
        file_path = src_path / file
        if file_path.exists():
            print(f"  ✅ {file}")
        else:
            print(f"  ❌ {file} - MISSING")
else:
    print(f"  ❌ src/ directory missing")

print("\n🎉 Validation complete!")
print("\n📖 Next steps:")
print("1. Verify data is in data/ directory")
print("2. Activate pyscenic environment")
print("3. Start with Stage 1: cd 01_pyscenic_runs")
print("4. Follow the README.md instructions")